In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_credentials = {
    "instance_id": "openshift",
    "url": "https://zen-cpd-zen.apps.icp4d-experiences-lite-xx.demo.ibmcloud.com",
    "username":"admin",
    "password": "password",
    "version": "2.5.0"
}
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
wml_client.version

In [ ]:
space_name = "OpenScale Deployment Space"
spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in spaces:
    if space['entity']['name'] == space_name:
        space_id = space["metadata"]["guid"]
wml_client.default_space_id = space_id

In [ ]:
deployment_details = None
for resource in wml_client.deployments.get_details()['resources']:
    if resource['entity']['name'] == 'Mortgage Default - OpenScale':
        deployment_details = resource
scoring_uid = deployment_details['metadata']['guid']
scoring_uid

In [ ]:
!rm mortgage_feed.json
!wget https://raw.githubusercontent.com/emartensibm/mortgage-default/master/mortgage_feed.json

In [ ]:
import json

with open('mortgage_feed.json', 'r') as scoring_file:
    data = json.load(scoring_file)
data['fields'][1:]

In [ ]:
scoring_payload = {wml_client.deployments.ScoringMetaNames.INPUT_DATA:
                   [
                       {
                           "fields": data['fields'][1:],
                           "values": [],
                           "meta": {
                               "fields": ["ID"],
                               "values": []
                           }
                       }
                    ]
                  }

In [ ]:
import random
import string

letters = string.digits

for _ in range(0, 101):
    value_to_score = random.choice(data['values'])
    scoring_payload['input_data'][0]['values'].append(value_to_score[1:])
    scoring_payload['input_data'][0]['meta']['values'].append([int(''.join(random.choices(letters, k=8)))])
print(len(scoring_payload['input_data'][0]['values']))

In [ ]:
predictions = wml_client.deployments.score(scoring_uid, scoring_payload)
print(predictions)